In [2]:
import xmltodict
from bs4 import BeautifulSoup
import requests
import string
import nltk
import gensim
response = requests.get('http://export.arxiv.org/api/query?search_query=all:corr&amp;id_list=&amp;start=0&amp;max_results=25')
doc = xmltodict.parse(response.text)
#d = doc['rss']['channel']['item']
d = doc['feed']['entry']
docs = list()
for i in d:
    docs.append(i['summary'])

from nltk.tokenize import word_tokenize
#return tokens
token_docs = [word_tokenize(i.encode('ascii',errors='ignore').decode()) for i in docs]

word_docs = list()
#remove punctuation
for token_doc in token_docs:
    word_docs.append([word for word in token_doc if word.isalpha()])
no1ltw_docs = list()
#remove 1 letter words and lowercase
for word_doc in word_docs:
    no1ltw_docs.append([w.lower() for w in word_doc if not len(w) <= 2])

bigram = gensim.models.Phrases(no1ltw_docs, min_count=1, threshold=1)
bigram_mod = gensim.models.phrases.Phraser(bigram)

bigram_docs = list()
for no1ltw_doc in no1ltw_docs:
    bigram_docs.append(bigram_mod[no1ltw_doc])
    
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

id2word = corpora.Dictionary(bigram_docs)
corpus = [id2word.doc2bow(text) for text in bigram_docs]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=10,chunksize = 50,update_every=1,
                                           passes=10000,alpha='auto',per_word_topics=True)
from pprint import pprint
pprint(lda_model.print_topics())
# Compute Perplexity
print('\nModel Perplexity: ', lda_model.log_perplexity(corpus))  #lower is better
coherence_model_lda = CoherenceModel(model=lda_model, texts=bigram_docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nModel Coherence: ', coherence_lda)#higher is better


[(0,
  '0.031*"and" + 0.022*"partial_corr" + 0.018*"identify" + 0.014*"missing" + '
  '0.014*"patterns" + 0.014*"the" + 0.009*"for" + 0.009*"graphical" + '
  '0.009*"insights_into" + 0.009*"temperature_partial"'),
 (1,
  '0.047*"the" + 0.020*"with" + 0.016*"and" + 0.016*"for" + 0.012*"that_the" + '
  '0.012*"that" + 0.012*"space" + 0.012*"such" + 0.008*"are" + '
  '0.008*"composition"'),
 (2,
  '0.057*"the" + 0.038*"and" + 0.019*"corr" + 0.012*"that" + 0.012*"data" + '
  '0.012*"architecture" + 0.012*"this" + 0.012*"repository" + 0.008*"for" + '
  '0.008*"with_the"'),
 (3,
  '0.040*"the" + 0.026*"and" + 0.015*"for_the" + 0.015*"accuracy" + '
  '0.011*"dblp" + 0.011*"arabic" + 0.011*"research" + 0.011*"languages" + '
  '0.011*"with" + 0.011*"recognition"'),
 (4,
  '0.017*"representation" + 0.017*"the" + 0.017*"that" + 0.011*"for" + '
  '0.011*"show" + 0.011*"algorithm" + 0.011*"the_memory" + 0.011*"and" + '
  '0.011*"representations" + 0.006*"relatively"'),
 (5,
  '0.032*"the" + 0.019*"